In [1]:
import pandas as pd
import torch
import mysql.connector
import dgl.data
import numpy as np
from dgl import save_graphs, load_graphs
import torch as th
import gensim
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
import requests
import time
#from transformers import *

#tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
#model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')

[nltk_data] Downloading package punkt to /home/david/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/david/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
cnx = mysql.connector.connect(user='david', password='daviddung1993',
                          host='127.0.0.1',
                          database='computervision')
cursor = cnx.cursor()
headers = {"x-api-key": "M7HSjQNeTfai6l7JUiDZB8XYc85BHnHt3R0NXSEd"}

In [3]:
test_papers = []
with open(r'./test_papers.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        test_papers.append(x)
in_params_test = ','.join(['%s'] * len(test_papers))

train_papers = []
with open(r'./train_papers.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        train_papers.append(x)

total_papers = train_papers + test_papers
in_params_train = ','.join(['%s'] * len(train_papers))

val_papers = []
with open(r'./val_papers.txt', 'r') as fp:
    for line in fp:
        x = line[:-1]
        val_papers.append(x)

in_params_val = ','.join(['%s'] * len(val_papers))
in_params = ','.join(['%s'] * len(train_papers + test_papers))

In [4]:
def remap(x, mapping):
    return mapping.get(x, x)

In [14]:
cursor.execute("select distinct p2.PaperID from Papers p, authoredBy b, authoredBy b2, Papers p2 where p.PaperID in (%s) and p.PaperID = b.PaperID and b.AuthoredByID = b2.AuthoredByID and b2.PaperID = p2.PaperID and p2.Pub_Year != 2022" % in_params_test, test_papers)
train_papers = [x[0] for x in cursor.fetchall()]
total_papers = train_papers + test_papers
in_params = ','.join(['%s'] * len(total_papers))

In [15]:
cursor.execute("select b.ReferenceID, b.ReferencedByID from referencedBy b, Papers p where b.ReferencedByID in (%s) and b.ReferenceID = p.PaperID and p.`primary author` in (select AuthorID from affiliatedTo) and b.ReferenceID in (select PaperID from Papers) and b.ReferencedByID in (select PaperID from Papers)" % in_params, total_papers)
paper_edges = np.array(cursor.fetchall())
all_papers = np.unique(paper_edges)

remapped_papers = {all_papers[i]: i  for i in range(len(all_papers))}
vec_remap = np.vectorize(remap, otypes=[str])
paper_edges = vec_remap(paper_edges, remapped_papers)

all_papers_list = all_papers.tolist()
all_params = ','.join(['%s'] * len(all_papers_list))

#cursor.execute("select p.PaperID, p.`primary author`, a.Gender, p.Title from Papers p, Authors a where p.PaperID in (%s) and p.`primary author` in (select AuthorID from affiliatedTo) and p.`primary author` = a.AuthorID" % all_params, all_papers_list)
cursor.execute("select p.PaperID, a.AuthorID, a.Gender, p.Title from Papers p, Authors a, authoredBy ab where p.PaperID in (%s) and p.PaperID = ab.PaperID and ab.AuthoredByID = a.AuthorID" % all_params, all_papers_list) #and a.AuthorID in (select AuthorID from affiliatedTo)
author_edges = np.array(cursor.fetchall())

all_authors = np.unique(author_edges[:,1])
remapped_authors = {all_authors[i]: i  for i in range(len(all_authors))}

author_edges[:,1] = vec_remap(author_edges[:,1], remapped_authors)
author_edges[:,0] = vec_remap(author_edges[:,0], remapped_papers)

In [17]:
model_path = './GoogleNews-vectors-negative300.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

In [19]:
def title_to_vec(title):
    title = title.replace("-", " ")
    tokens = nltk.word_tokenize(title)
    tokens = [token for token in tokens if token.lower() not in stop_words]
    vectors = []
    for word in tokens:
        try:
            vec = model[word.lower()]
            vectors.append(vec)
        except KeyError:
            #print(word)
            continue
    title_vector = np.zeros(300)
    try:
        title_vector = sum(vectors) / len(vectors)
    except ZeroDivisionError:
        pass
    return title_vector

In [20]:
all_papers_info = pd.DataFrame(author_edges)
all_papers_info[0] = all_papers_info[0].astype(int)
all_papers_info = all_papers_info.sort_values(0)
paper_feats = []
stop_words = set(stopwords.words('english'))
for title in all_papers_info[3]:
    title_vec = title_to_vec(title)
    paper_feats.append(title_vec)
paper_feats = torch.tensor(paper_feats)

/tmp/ipykernel_9780/2618123609.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  paper_feats = torch.tensor(paper_feats)


In [23]:
headers = {"x-api-key": "M7HSjQNeTfai6l7JUiDZB8XYc85BHnHt3R0NXSEd"}

In [24]:
def request_author_information(authors):
    resp = requests.post("https://api.semanticscholar.org/graph/v1/author/batch?fields=papers.title,affiliations,paperCount,citationCount,hIndex", headers=headers, json={"ids": authors}).json()
    return resp

def request_author_individual_dfinformation(author):
    resp = requests.get(f"https://api.semanticscholar.org/graph/v1/author/{author}?fields=papers.title,paperCount,citationCount,hIndex", headers=headers).json()
    return resp

def convert_history_to_vector(response):
    if not response:
        author_hist = np.zeros(300)
        author_hist = np.append(author_hist, [0,0,0])
        return author_hist

    author_papers = response["papers"]
    author_hist = []

    for paper in author_papers:
        paperID = paper["paperId"]
        if paperID in test_papers:
            continue
        title = paper["title"]
        title_vect = title_to_vec(title)
        if np.count_nonzero(title_vect) > 0:
            author_hist.append(title_vect)
    if len(author_hist) != 0:
        author_hist = np.mean(np.array(author_hist), axis=0)
    else:
        author_hist = np.zeros(300)

    author_paperCount = response["paperCount"]
    author_citationCount = response["citationCount"]
    author_hIndex = response["hIndex"]
    author_hist = np.append(author_hist, [author_paperCount, author_citationCount, author_hIndex])

    return author_hist

In [30]:
all_authors[101791]

'47399096'

In [29]:
np.where(all_authors == '47399096')

(array([101791]),)

In [39]:
all_authors.astype(int)

array([10001427, 10007124, 10007273, ...,  9994893,  9995883,  9998857])

In [40]:
np.savetxt('all_authors.txt', all_authors.astype(int))